# NLP Basics

## Basic NLP Practice (1)

Install and Load Libraries

In [2]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post7-py3-none-any.whl size=2950 sha256=cc7078e2450840d1b293036933f73cc1469c0fca436330c97a3a2ef895238dde
  Stored in directory: /root/.cache/pip/wheels/c8/9c/85/72901eb50bc4bc6e3b2629378d172384ea3dfd19759c77fd2c
Successfully built sklearn


In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split, cross_val_score

Reading the data file

In [5]:
spam_text = pd.read_csv('SMSSpamCollection', delimiter='\t', header = None)
print(spam_text.head())

      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [6]:
#Getting the count on the repeated words
print('Number of spam messaged:\n',spam_text[spam_text[0]== 'spam'][0].count())
print('Number of ham messaged:\n',spam_text[spam_text[0]== 'ham'][0].count())

Number of spam messaged:
 747
Number of ham messaged:
 4825


In [7]:
#Setting the dependent variable and independent variable
X = spam_text[1].values
Y = spam_text[0].values

In [8]:
#Split data for train and test
X_train_raw, X_test_raw, Y_train, Y_test = train_test_split(X,Y)

In [9]:
#Transform the text into vectors
vect = TfidfVectorizer()
X_train = vect.fit_transform(X_train_raw)
X_test = vect.fit_transform(X_test_raw)

In [10]:
#Create a Classifier choosing Logistic Regression
classifier = LogisticRegression()
classifier.fit(X_train,Y_train)

LogisticRegression()

In [14]:
#Save predictions based on the test data
predictions = classifier.predict(X_test)

ValueError: ignored

In [15]:
for i, prediction in enumerate(predictions[:5]):
    print('Predicted: %s, message: %s --- ture label: %s' % (prediction, X_test_raw[i], y_test[i]))

NameError: ignored

## Basic NLP Practice (1)

## Load Libraries

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

## Load and read the data

In [18]:
df = pd.read_csv('train.tsv', header=0, delimiter='\t')
print(df)

        PhraseId  SentenceId  \
0              1           1   
1              2           1   
2              3           1   
3              4           1   
4              5           1   
...          ...         ...   
156055    156056        8544   
156056    156057        8544   
156057    156058        8544   
156058    156059        8544   
156059    156060        8544   

                                                   Phrase  Sentiment  
0       A series of escapades demonstrating the adage ...          1  
1       A series of escapades demonstrating the adage ...          2  
2                                                A series          2  
3                                                       A          2  
4                                                  series          2  
...                                                   ...        ...  
156055                                          Hearst 's          2  
156056                          forced avuncula

## EDA

In [19]:
df.count()

PhraseId      156060
SentenceId    156060
Phrase        156060
Sentiment     156060
dtype: int64

In [20]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [22]:
df['Phrase'].head()

0    A series of escapades demonstrating the adage ...
1    A series of escapades demonstrating the adage ...
2                                             A series
3                                                    A
4                                               series
Name: Phrase, dtype: object

In [23]:
df['Sentiment'].describe()

count    156060.000000
mean          2.063578
std           0.893832
min           0.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           4.000000
Name: Sentiment, dtype: float64

In [24]:
print(df['Sentiment'].value_counts())

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64


In [25]:
df['Sentiment'].value_counts()/df['Sentiment'].count()

2    0.509945
3    0.210989
1    0.174760
4    0.058990
0    0.045316
Name: Sentiment, dtype: float64

## Model Selection

In [29]:
#Define independet and dependet variables
X, Y = df['Phrase'], df['Sentiment']

In [27]:
#Split data
 X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.5)

In [32]:
#create a pipeline to vectorise the text data
pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression())
])

In [33]:
#Specify Parameters
parameters = {
    'vect__max_df': (0.25, 0.5),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__use_idf': (True, False),
    'clf__C': (0.1, 1, 10),
}

In [36]:
#Use grid Search and the vectorisation pipeline
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': (0.1, 1, 10), 'vect__max_df': (0.25, 0.5),
                         'vect__ngram_range': ((1, 1), (1, 2)),
                         'vect__use_idf': (True, False)},
             scoring='accuracy', verbose=1)

In [ ]:
print('Best score: %0.3f' % grid_search.best_score_)

In [ ]:
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('t%s: %r' % (param_name, best_parameters[param_name]))

In [ ]:
predictions = grid_search.predict(X_test)

In [ ]:
print('Accuracy: %s' % accuracy_score(y_test, predictions))

In [ ]:
print('Confusion Matrix:')
print(confusion_matrix(y_test, predictions))

## Saving the grid search object

In [ ]:
 type(grid_search)

In [ ]:
sklearn.model_selection._search.GridSearchCV

In [ ]:
grid_search.cv_results_

In [ ]:
import pickle
pickle.dump(grid_search, open("./nlp_basics_2_grid_search.pkl", "wb"))

## Basic NLP & GridSearch Practice (1)

## import library

In [ ]:
import pickle

## Import the GridSearch file

In [ ]:
grid_search_loaded = pickle.load(open("./nlp_basics_2_grid_search.pkl", "rb"))

## Load the saved results from the gridsearch

In [ ]:
grid_search_loaded.cv_results_

## Save & Load a model

### import libraries

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

In [ ]:
digits = datasets.load_digits()

X = digits.data
y = digits.target
X_train, X_test,\
y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [ ]:
clf = SGDClassifier(max_iter=1000, eta0=0.001)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)